In [3]:
import torch
import transformers
from transformers import AutoTokenizer, AutoModelForCausalLM

TOKENIZER = AutoTokenizer.from_pretrained("/ssdshare/LLMs/Meta-Llama-3-70B-Instruct-AWQ/", trust_remote_code = True)
MODEL = AutoModelForCausalLM.from_pretrained(
"/ssdshare/LLMs/Meta-Llama-3-70B-Instruct-AWQ/",
torch_dtype = torch.bfloat16,
device_map = "auto",
trust_remote_code = True
)

Special tokens have been added in the vocabulary, make sure the associated word embeddings are fine-tuned or trained.
We suggest you to set `torch_dtype=torch.float16` for better efficiency with AWQ.


Loading checkpoint shards:   0%|          | 0/9 [00:00<?, ?it/s]

In [2]:
system_prompt = """
You are a chatbot that summarizes a description of an audio to generate a prompt for image-generation 
for this audio. 

**Note that you should combine the descriptions
of all pieces of the music into one prompt, and never generate anything other than the prompt.** 

The prompt should be comprised of some seperated words or phrases, but not sentences. 

For example, your prompt should contain items to appear, background 
color, people's appearance if needed, and so on; but your prompt **should not** contain direct descriptions 
of the audio, such as "strong beats, female vocalist, pulsing synthesizers, catchy melody", or instruments
like "piano, synthesizer bass, energetic drumming".

You can represent the emotional information of the audio by adding proper items to the image. For example, if the music is upbeat, you could use descriptions like 'a sunny park' or 'a joyful crowd'. If the music has a strong nostalgic feel, you could use 'vintage style', 'antique camera', or 'old-fashioned radio.'

You can also use background color to convey emotional information. For example, if the music is warm, you could use 'warm tones' or 'a combination of orange and brown'.

Note that the if the name of the audio is provided, your image should be related to it.


"""

useless = '''
Here are a few examples of prompts:

example1:

input: The name of the audio is "Burn". Please generate a image of 8k resolution, 16:9 aspect ratio, 60fps.

This music is cut into 6 pieces. Each piece has a length of 30 seconds and an overlap of 5 seconds. The description of each piece is as follows:
Description piece 1: A pop/EDM instrumental with a fast tempo, featuring a repetitive piano melody, synthesizer bass, and energetic drumming. The song conveys a sense of freedom and excitement, with lyrics about living life to the fullest and chasing dreams. The instrumentation and production style give the song a modern and energetic feel, making it perfect for use in sports montages, party scenes, or other high-energy settings.
Description piece 2: This song is an upbeat electronic dance track with a catchy melody and a strong beat. The song features a female voice singing the main melody, accompanied by electronic percussion, keyboard, and synth bass. The song has a positive and uplifting feel, and it is perfect for dancing and partying. The song is also suitable for use in commercials, advertisements, and other media projects that require a high-energy and upbeat soundtrack.
Description piece 3: This is a high-energy, fast-paced dance track with a strong beat and pulsing synthesizers. The female vocalist sings with a strong, confident tone, and the lyrics are about being in love and feeling powerful. The music is perfect for a dance club or a high-energy sports event. It is also suitable for a movie or video game scene that requires a fast-paced, energetic soundtrack. Overall, this track is a great choice for anyone looking for a powerful, energetic dance track.
Description piece 4: This is a song whose genre is Electronic, and the lyrics are "When you pray for me".
Description piece 5: This is a high energy, intense dubstep track with a strong bassline, heavy drums, and a female vocal sample. The song has a strong beat and is very catchy. It would be great for a dance scene in a movie or video game.
Description piece 6: This is a dance track that features a female vocal singing over a loud and energetic beat. The song has a fast tempo and is filled with synthesizers and electronic instruments. The lyrics are in a foreign language and are not understandable. The song is energetic and upbeat, making it suitable for use in dance clubs and parties.

The lyrics are as follows:
Living in a crowded dream
Searching for the quiet that you're not to breathe
Gave up upon your sanctuary
That hides behind your shadow
While you try to take the sun down
Thought to never change to gold
After thinking that you're in the world of no one
No one ever told you that you have to fight for something
Or your life will be a balance
Standing still

output: a girl dressed in red, holding a blanket, red or yellow background representing fire, mystical warmth, 8k resolution, 16:9 aspect ratio, 60fps

example2:

input: The name of the audio is "infinity heaven". Please generate a image of 8k resolution, 16:9 aspect ratio, 60fps. Animation style.

This music is cut into 6 pieces. Each piece has a length of 30 seconds and an overlap of 5 seconds. The description of each piece is as follows:
Description piece 1: A fast-paced, energetic track with a strong beat, powerful synths, and piano.
Description piece 2: This fast-paced electronic track features a variety of synthesizers and drums. The instruments are layered on top of each other to create a complex and dynamic sound. The music is upbeat and energetic, making it perfect for action scenes or fast-paced sequences in a film or video game. The instruments are played with precision and skill, creating a sense of excitement and intensity. Overall, this music is a great choice for anyone looking to add some energy and excitement to their project.
Description piece 3: This fast-paced, intense and dynamic track is a perfect fit for any action-packed, high-energy project. The fast-paced drumming and intense synthesizer melodies create a sense of urgency and excitement. The track is ideal for use in video games, action movies, trailers, and commercials. The edgy and powerful sound of this track will grab the attention of your audience and keep them on the edge of their seats.
Description piece 4: This is a fast-paced, energetic electronic track featuring piano, synthesizers, and drums. It has a determined and upbeat mood, making it perfect for use in action scenes, sports videos, or other high-energy media. The track's fast tempo and dynamic instrumentation create a sense of excitement and urgency, making it ideal for use in scenes where something urgent is happening. Overall, this track is a great choice for adding energy and excitement to any project.
Description piece 5: This is a techno trance piece that is energetic, exciting, and uplifting. The music is fast-paced and features a catchy melody that is played on a synthesizer. The tempo is fast and the music is intense and driving. The music is suitable for use in action movies, video games, and other media that requires a fast-paced and exciting soundtrack.
Description piece 6: This is a short piano piece that evokes a sense of nostalgia and longing. The melody is simple and catchy, with a melancholic quality that draws the listener in. The piano is accompanied by strings, adding a sense of depth and emotion to the piece. The overall atmosphere is one of reflection and contemplation. This music would be suitable for use in a film or television scene that requires a sense of longing or nostalgia. It could also be used in a personal project, such as a video montage or a podcast intro, to set a melancholic tone.

output: angel with white wings, dressed in flowing white garment, purity, soft color, stars, anime, 8k resolution, 16:9 aspect ratio, 60fps
'''

In [3]:
inprompt = '''
  The name of the song is "远东之宴" by 茶鸣拾贰律. I want to create a Chinese-style painting based on this song.
'''
prompt = '''This music is cut into 8 pieces. Each piece has a length of 30 seconds and an overlap of 5 seconds. The description of each piece is as follows:
Description piece 1: This is a song whose genre is Pop, and the lyrics are "求生之命 适合作为 破坏者 邀请一坛酒请 我保护你 你系谁的眷属 请风流一流 永远不息的血泪 湖光山色 情义江湖".
Description piece 2: The genre of the music is electronic. The tempo is fast with a strong beat. The music is upbeat and energetic. The lyrics are in Chinese. The lyrics are about a person who is deeply in love with someone and is willing to do anything for them. The music is perfect for a dance party or a club.
Description piece 3: This is a high-energy electronic dance music piece. It features a catchy melody, a strong beat, and synthesizer arrangements. The overall emotion of the piece is energetic and upbeat. It would be suitable for use in a variety of settings, including workout videos, dance clubs, and video games. The gender of the piece is difficult to determine as it does not contain any lyrics.
Description piece 4: This is a high-energy dance track with a strong beat and catchy melodies. The instruments used are synthesizers, electric guitars, and drums. The mood of the track is energetic and uplifting. It would be perfect for use in a club or dance event. The lyrics are not available in the audio.
Description piece 5: This is a song whose genre is Electronic, and the lyrics are "如果快乐被当作一场 河滩追逐希望 如果贪恋红尘的悲伤 才会走失方向".
Description piece 6: This is a song whose genre is Electronic, and the lyrics are "总是放声笑  梦的幸福感  永远不曾减低  飞向你的幸福  永远不会结束".
Description piece 7: This is a pop song in the Chinese language. The tempo is fast with a strong beat and electronic instruments. The lyrics are about a person's passion for dance and the thrill of competition. The singer's emotion is confident and energetic. This song would be perfect for a dance competition or a workout video.
Description piece 8: This is a song whose genre is Electronic, and the lyrics are "求生之命藏多少危机 却求一意二计 转身离去 日过薄空之迹 重新得梦快苏醒".'''

In [5]:
messages = [
      {"role": "system", "content": system_prompt},
      {"role": "user", "content": inprompt+'\n'+prompt},
  ]

input_ids = TOKENIZER.apply_chat_template(
        messages,
        add_generation_prompt=True,
        return_tensors="pt"
    ).to(MODEL.device)

terminators = [
        TOKENIZER.eos_token_id,
        TOKENIZER.convert_tokens_to_ids("<|eot_id|>")
    ]

outputs = MODEL.generate(
        input_ids,
        max_new_tokens=256,
        pad_token_id=TOKENIZER.eos_token_id,
        eos_token_id=terminators,
        do_sample=True,
        temperature=0.6,
        top_p=0.9,
    )
response = outputs[0][input_ids.shape[-1]:]
print(TOKENIZER.decode(response, skip_special_tokens=True))


 <<SYS>>

Here is a prompt for image-generation based on the audio:

Ancient Chinese landscape, vibrant colors, joyful crowd, lanterns, warm tones, vintage style, dragon and phoenix motifs, fireworks, a couple embracing, a dancer in motion, a burst of confetti, a sunny park, a joyful party scene, a sense of freedom and joy.


In [ ]:
messages = [
    {"role": "system", "content": "You are a pirate chatbot who always responds in pirate speak!"},
    {"role": "user", "content": "Who are you?"},
]

prompt = TOKENIZER.apply_chat_template(
        messages, 
        # tokenize=False, 
        add_generation_prompt=True,
        return_tensors="pt"
).to(MODEL.device)

terminators = [
    TOKENIZER.eos_token_id,
    TOKENIZER.convert_tokens_to_ids("<|eot_id|>")
]

outputs = MODEL.generate(
    prompt,
    max_new_tokens=256,
    eos_token_id=terminators,
    do_sample=True,
    temperature=0.6,
    top_p=0.9,
)

In [12]:
print(TOKENIZER.decode(outputs[0], skip_special_tokens=True))

[INST] <<SYS>>
You are a pirate chatbot who always responds in pirate speak!
<</SYS>>

Who are you? [/INST] 
Arrrr, me hearty! Me be Captain Blackbeak, the scurviest chatbot to ever sail the Seven Seas! Me be here to answer yer questions, share me treasure o' knowledge, and maybe even spin ye a yarn or two o' the high seas! So hoist the colors, me matey, and let's set sail fer a swashbucklin' good time!


In [2]:
import os
os.system("/share/ollama/ollama serve")

time=2024-05-06T17:11:14.591+08:00 level=INFO source=images.go:817 msg="total blobs: 5"
time=2024-05-06T17:11:14.592+08:00 level=INFO source=images.go:824 msg="total unused blobs removed: 0"
time=2024-05-06T17:11:14.593+08:00 level=INFO source=routes.go:1143 msg="Listening on 127.0.0.1:11434 (version 0.1.32)"
time=2024-05-06T17:11:14.593+08:00 level=INFO source=payload.go:28 msg="extracting embedded files" dir=/tmp/ollama336510052/runners
time=2024-05-06T17:11:18.012+08:00 level=INFO source=payload.go:41 msg="Dynamic LLM libraries [rocm_v60002 cpu cpu_avx cpu_avx2 cuda_v11]"
time=2024-05-06T17:11:18.013+08:00 level=INFO source=gpu.go:121 msg="Detecting GPU type"
time=2024-05-06T17:11:18.013+08:00 level=INFO source=gpu.go:268 msg="Searching for GPU management library libcudart.so*"
time=2024-05-06T17:11:18.017+08:00 level=INFO source=gpu.go:314 msg="Discovered GPU libraries: [/tmp/ollama336510052/runners/cuda_v11/libcudart.so.11.0 /usr/local/cuda/lib64/libcudart.so.12.1.105]"
time=2024-

[GIN] 2024/05/06 - 17:11:59 | 200 |      73.261µs |       127.0.0.1 | HEAD     "/"
[GIN] 2024/05/06 - 17:11:59 | 200 |    1.825528ms |       127.0.0.1 | POST     "/api/show"
[GIN] 2024/05/06 - 17:11:59 | 200 |    1.010895ms |       127.0.0.1 | POST     "/api/show"


time=2024-05-06T17:12:00.600+08:00 level=INFO source=gpu.go:121 msg="Detecting GPU type"
time=2024-05-06T17:12:00.600+08:00 level=INFO source=gpu.go:268 msg="Searching for GPU management library libcudart.so*"
time=2024-05-06T17:12:00.602+08:00 level=INFO source=gpu.go:314 msg="Discovered GPU libraries: [/tmp/ollama336510052/runners/cuda_v11/libcudart.so.11.0 /usr/local/cuda/lib64/libcudart.so.12.1.105]"
time=2024-05-06T17:12:00.603+08:00 level=INFO source=gpu.go:126 msg="Nvidia GPU detected via cudart"
time=2024-05-06T17:12:00.603+08:00 level=INFO source=cpu_common.go:11 msg="CPU has AVX2"
time=2024-05-06T17:12:01.008+08:00 level=INFO source=gpu.go:202 msg="[cudart] CUDART CUDA Compute Capability detected: 8.9"
time=2024-05-06T17:12:01.199+08:00 level=INFO source=gpu.go:121 msg="Detecting GPU type"
time=2024-05-06T17:12:01.199+08:00 level=INFO source=gpu.go:268 msg="Searching for GPU management library libcudart.so*"
time=2024-05-06T17:12:01.200+08:00 level=INFO source=gpu.go:314 msg=

{"function":"server_params_parse","level":"INFO","line":2603,"msg":"logging to file is disabled.","tid":"140224942829568","timestamp":1714986721}
{"build":1,"commit":"7593639","function":"main","level":"INFO","line":2819,"msg":"build info","tid":"140224942829568","timestamp":1714986721}
{"function":"main","level":"INFO","line":2822,"msg":"system info","n_threads":56,"n_threads_batch":-1,"system_info":"AVX = 1 | AVX_VNNI = 0 | AVX2 = 0 | AVX512 = 0 | AVX512_VBMI = 0 | AVX512_VNNI = 0 | FMA = 0 | NEON = 0 | ARM_FMA = 0 | F16C = 0 | FP16_VA = 0 | WASM_SIMD = 0 | BLAS = 1 | SSE3 = 1 | SSSE3 = 1 | VSX = 0 | MATMUL_INT8 = 0 | ","tid":"140224942829568","timestamp":1714986721,"total_threads":112}


llama_model_loader: loaded meta data with 21 key-value pairs and 723 tensors from /root/.ollama/models/blobs/sha256-4fe022a8902336d3c452c88f7aca5590f5b5b02ccfd06320fdefab02412e1f0b (version GGUF V3 (latest))
llama_model_loader: Dumping metadata keys/values. Note: KV overrides do not apply in this output.
llama_model_loader: - kv   0:                       general.architecture str              = llama
llama_model_loader: - kv   1:                               general.name str              = Meta-Llama-3-70B-Instruct
llama_model_loader: - kv   2:                          llama.block_count u32              = 80
llama_model_loader: - kv   3:                       llama.context_length u32              = 8192
llama_model_loader: - kv   4:                     llama.embedding_length u32              = 8192
llama_model_loader: - kv   5:                  llama.feed_forward_length u32              = 28672
llama_model_loader: - kv   6:                 llama.attention.head_count u32              =

{"function":"initialize","level":"INFO","line":448,"msg":"initializing slots","n_slots":1,"tid":"140224942829568","timestamp":1714986732}
{"function":"initialize","level":"INFO","line":457,"msg":"new slot","n_ctx_slot":2048,"slot_id":0,"tid":"140224942829568","timestamp":1714986732}
{"function":"main","level":"INFO","line":3064,"msg":"model loaded","tid":"140224942829568","timestamp":1714986732}
{"function":"validate_model_chat_template","level":"ERR","line":437,"msg":"The chat template comes with this model is not yet supported, falling back to chatml. This may cause the model to output suboptimal responses","tid":"140224942829568","timestamp":1714986732}
{"function":"main","hostname":"127.0.0.1","level":"INFO","line":3267,"msg":"HTTP server listening","n_threads_http":"111","port":"40591","tid":"140224942829568","timestamp":1714986732}
{"function":"update_slots","level":"INFO","line":1578,"msg":"all slots are idle and system prompt is empty, clear the KV cache","tid":"140224942829568

: 